# part1

In [2]:
#importing libraries
import pandas as pd
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

In [3]:
df=pd.read_csv("SPAM text message 20170820 - Data.csv")

In [4]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df.shape

(5572, 2)

In [6]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [8]:
df.isnull().sum()


Category    0
Message     0
dtype: int64

# PART2

In [9]:
####text preprocessing#####

def preprocessing(txt):
    txt=re.sub('[^a-zA-Z]',' ',txt)
    txt=str.lower(txt)
    txt=nltk.word_tokenize(txt)#tokenise
    txt=[p.stem(word) for word in txt if not word in set(stopwords.words('english')) ]#stemming and stopwords removal
    txt=[word for word in txt if len(word) >2]
    txt=' '.join(txt)
    return txt

In [10]:
p=PorterStemmer()
df['filtered']=df['Message'].apply(preprocessing)

In [11]:
df.head()

,Category,Message,filtered
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazi avail bugi great world buff...
1,ham,Ok lar... Joking wif u oni...,lar joke wif oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entri wkli comp win cup final tkt may tex...
3,ham,U dun say so early hor... U c already then say...,dun say earli hor alreadi say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think goe usf live around though


# part3

In [12]:
#####bag of word model####
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000)
cleaned=cv.fit_transform(df['filtered']).toarray()

In [13]:
cleaned.shape

(5572, 5000)

In [14]:
####handling categorical variable
y=pd.get_dummies(df['Category'])
y=y.iloc[:,1].values

In [15]:
###splillitng dataset
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(cleaned,y,test_size=0.2,random_state=0)

In [16]:
print(xtrain.shape,ytrain.shape)
print(xtest.shape,ytest.shape)

(4457, 5000) (4457,)
(1115, 5000) (1115,)


# part4

In [17]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB().fit(xtrain,ytrain)

In [18]:
yhash=model.predict(xtest)

In [19]:
yhash.shape

(1115,)

In [20]:
from sklearn.metrics import confusion_matrix
con=confusion_matrix(ytest,yhash)
con

array([[942,  13],
       [  6, 154]])

In [21]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(ytest,yhash)
acc

0.9829596412556054